# Spine Model


In [1]:
from numpy import *   #ones, cumsum, sum, isscalar
from matplotlib.pylab import * 
%matplotlib inline
plotose = True
import pickle

import PS_lib as lb
import PS_storage as st
from bootstrap import btset_cd, generateplots, btShInh


# Parameter definitions
# Data is stored here      
data = st.dataStorage() # some default parameters defined.
data.dt = 0.05
data.NMDA = False

# Definition of the model.
lb.h.dt = data.dt
NMDA = data.NMDA
model = lb.loadNeuron("Basic2.hoc",axon=False)

# Adding piece of dendritic branch with spine
model.addDend(name="DendE",locus="dendA1",L=1.0,D=1.5,ilocus=1)
model.addSpne(locus="DendE",ilocus=0.5,L=1.0,D=1.0,Lneck=1.0,Dneck=0.15)
# Temperature of the neuron
lb.h.celsius = model.temperature
model.addDend(name="DendN",locus=3,L=240.0,D=0.5,ilocus=1)
data.model = model.__dict__

model.soma.L = 20
model.dend[-1].L = 200
model.dend[-1].diam = 10
model.addSpne(locus="DendE",ilocus=0.5,L=1.0,D=1.0,Lneck=1.0,Dneck=0.15)

print(model.soma.L,model.soma.diam,model.soma(0.5).area(),model.dend[-1](0.5).area())

print("Topology:")
print(lb.h.topology())


	0 
	1 
	1 
	1 
	1 
20.0 29.78380012512207 1871.3713533814052 6283.185307179586
Topology:

|-|       soma(0-1)
   `|       dendA1c(0-1)
     `|       dendA1(0-1)
       `|       DendE(0-1)
         `|       DendN(0-1)
        `|       NSpne000(0-1)
          `|       Spne000(0-1)
        `|       NSpne001(0-1)
          `|       Spne001(0-1)
   `|       dendB1(0-1)

1.0


### Recording places

We set recording vectors in all dendritic compartments, spine included

In [2]:
nsp = 500
dis = btset_cd(cd='DiS',size=nsp)
sis = btset_cd(cd='SiS',size=nsp)
sps = btset_cd(cd='Sp',size=nsp)

#generateplots(dis,corplot=True)

In [3]:
nsp = 500
#figure()
iPSDsh = btShInh(size=500)
#h = hist(iPSDsh,31)
#xlabel("iPSD shaft ($\mu m^2$)",fontsize=14)



In [4]:

rhGABAR = 500 # Receptors/um2
rhAMPAR = 1000 # Receptors/um2
rhNMDAR = 10 # Receptors/um2

gtrG = rhGABAR*30e-6*2/5.0
gtrA = rhAMPAR*11e-6*2/5.0
gtrN = rhNMDAR*30e-6/16.0

figureplot=False
if figureplot:
    fig, ax = subplots(nrows=1,ncols=4,figsize=(20,5))
    fig.suptitle("Number of Receptors",fontsize = 20)

    ax[0].hist(sis['A1']*rhAMPAR,31)
    ax[0].set_xlabel("#AMPAR in Sis")
    ax[1].hist(dis['A1']*rhAMPAR,31)
    ax[1].set_xlabel("#AMPAR in Dis")
    ax[2].hist(dis['A2']*rhGABAR,31)
    ax[2].set_xlabel("#GABAAR in Dis")
    ax[3].hist(iPSDsh*rhGABAR,31)
    ax[3].set_xlabel("#GABAAR in shaft")

    fig, ax = subplots(nrows=1,ncols=4,figsize=(20,5))
    fig.suptitle("Max Current",fontsize = 20)


    ax[0].hist(sis['A1']*gtrA/1e-6,31)
    ax[0].set_xlabel("I$_{max}$ (pA)")
    ax[1].hist(dis['A1']*gtrA/1e-6,31)
    ax[1].set_xlabel("I$_{max}$ (pA)")
    ax[2].hist(dis['A2']*gtrG/1e-6,31)
    ax[2].set_xlabel("I$_{max}$ (pA)")
    ax[3].hist(iPSDsh*gtrG/1e-6,31)
    ax[3].set_xlabel("I$_{max}$ (pA)")
    #savefig("D_maxI.png",bbox_inches='tight',pad_inches = 0)


In [5]:
for dend in model.dend:
    print(dend.L,dend.diam)

1.0 1.5
200.24984394500785 1.5000000000000002
50.15974481593781 1.5
1.0 1.5
200.0 10.0


In [6]:
sp0 = model.spne[0]
sp0.L = dis["L"][0]
sp0.diam = dis["D"][0]
sp1 = model.spne[1]
sp1.L = dis["L"][1]
sp1.diam = dis["D"][1]


### Excitatory synapse 

We need to define the arrival time and dynamics. These parameters may be modified later

In [7]:
lb.init_active(model, axon=False, soma=False, dend=False, dendNa=False,
                dendCa=False,spne=True)

In [8]:
model.AMPAlist = []
model.ncAMPAlist = []

tau1  = 0.5
tau2 = 3.0
nampa = 50
gmax = 15*nampa/1e6
nnmda = 2
gmaxN = 50*nnmda/1e6

AMPA0 = lb.h.Exp2Syn(1,sec = model.spne[0])
AMPA0.tau1 = tau1
AMPA0.tau2 = tau2

stimE0=lb.h.NetStim();stimE0.number = 1; 
NC0 = lb.h.NetCon(stimE0,AMPA0,0,0,gmax)
model.AMPAlist.append(AMPA0)
model.ncAMPAlist.append(NC0)
NC0.delay = 10

# NMDA part
lb.add_NMDAsyns(model, locs=[[0,0.5]], gmax=gmaxN,tau2=20.0)  
NMDA0 = model.NMDAlist[0]
NCN0 = model.ncNMDAlist[0]
stimN0=lb.h.NetStim();stimN0.number = 1;

In [9]:
NCN0 = lb.h.NetCon(stimN0,NMDA0,0,0,gmaxN)
model.ncNMDAlist[0] = NCN0
NMDA0.tau1 = 2.0
NMDA0.tau2 = 30.0

In [10]:
AMPA1 = lb.h.Exp2Syn(1,sec = model.spne[1])
AMPA1.tau1 = tau1
AMPA1.tau2 = tau2

stimE1=lb.h.NetStim();stimE1.number = 1; 
NC1 = lb.h.NetCon(stimE1,AMPA1,0,0,gmax)
model.AMPAlist.append(AMPA1)
model.ncAMPAlist.append(NC1)
NC1.delay = 10

# NMDA part
lb.add_NMDAsyns(model, locs=[[1,0.5]], gmax=gmaxN,tau2=20.0)  
NMDA1 = model.NMDAlist[0]
NCN1 = model.ncNMDAlist[0]
stimN1=lb.h.NetStim();stimN1.number = 1;


### Inhibitory synapse

In [11]:
nGABA = 35
gmaxG = nGABA*30e-6
GABA, NCG = lb.add_GABAsyns(model, locs=[[0,1]], spne=True, gmax=gmaxG,tau1=1.5,tau2=15.0)  


In [12]:
dendshaft = model.dend[-2]

nGABA = 35
gmaxG = nGABA*30e-6
GABAsh, NCGsh = lb.add_GABAsynscomp(model, dendshaft, loc = 0.5, gmax=gmaxG,tau1=1.5,tau2=20.0)  


In [13]:
model.dend

[dendA1c, dendA1, dendB1, DendE, DendN]

In [14]:
trec, vrec = lb.h.Vector(), lb.h.Vector()
gRec, iRec,  vspneRec = [], [], []
gNMDA_rec, iNMDA_rec = [], []
trec.record(lb.h._ref_t)
vrec.record(model.soma(0.5)._ref_v)


#n=0
vDendRec = []
caDendRec = []
#For all dendrites
for dend in model.dend:
    #Adding vectors for Voltage, and Calcium
    vDendRec.append(lb.h.Vector())
    caDendRec.append(lb.h.Vector())
    # Placing recording at mid-point in the dendritic branch
    vDendRec[-1].record(dend(0.5)._ref_v)
    # NO CALCIUM!?!?!
#Probably better to organize them in a dictionary        
vDendRec[1].record(model.dend[1](1.0)._ref_v)

# Spine voltage recording stuff
vspneRec.append(lb.h.Vector())
vspneRec.append(lb.h.Vector())
sp0 = model.spne[0]
vspneRec[0].record(sp0(0.5)._ref_v)
sp1 = model.spne[1]
vspneRec[1].record(sp1(0.5)._ref_v)

# Calcim

caDendRec = {}
caDendRec['sp0 iCa'] = lb.h.Vector()
caDendRec['sp1 iCa'] = lb.h.Vector()
caDendRec['sp0 Ca'] = lb.h.Vector()
caDendRec['sp1 Ca'] = lb.h.Vector()
caDendRec['sp0 iCa'].record(sp0(0.5)._ref_ica) 
caDendRec['sp1 iCa'].record(sp1(0.5)._ref_ica) 
caDendRec['sp0 Ca'].record(sp0(1.0)._ref_cai)
caDendRec['sp1 Ca'].record(sp1(1.0)._ref_cai) 


Vector[17]

In [15]:
vDendEL = []
dend = model.dend[1]
for i in range(10):
    vDendEL.append(lb.h.Vector())
    vDendEL[-1].record(dend((i+1.0)/10.0)._ref_v)

In [16]:
vDendEL2 = []
dend = model.dend[-1]
for i in range(10):
    vDendEL2.append(lb.h.Vector())
    vDendEL2[-1].record(dend((i+1.0)/10.0)._ref_v)

In [17]:
model.dend

[dendA1c, dendA1, dendB1, DendE, DendN]

In [18]:
iRec.append(lb.h.Vector())
iRec.append(lb.h.Vector())
currentAMPA = iRec[0]
currentGABA = iRec[1]

In [19]:
currentAMPA.record(AMPA0._ref_i)
currentGABA.record(GABA._ref_i)

Vector[39]

In [20]:
dendsizeL0 = 10
dendsizeL = max(max(max(dis['Dss'])+10,max(sis['Dss'])+10),180)
neck = model.neck[0]
Rneck = neck.L*1e-6/(neck.diam*1e-6/2.0)**2/pi*neck.Ra/100
dend = model.dend[0]
dend.L = dendsizeL0
dend.diam = 1.0
print(dend.L,dend.diam)
Rdend = dend.L*1e-6/(dend.diam*1e-6/2.0)**2/pi*dend.Ra/100
Rdendf = Rdend
dend = model.dend[1]
dendc = model.dend[1]
dendc.L = dendsizeL
dend.diam = 0.7
print(dend.L,dend.diam,Rdend/1e6)
Rdend +=dend.L*1e-6/2.0/(dend.diam*1e-6/2.0)**2/pi*dend.Ra/100
dend = model.dend[-2]
dend.diam = 0.7
dendsh = model.dend[-2]
dend.L = 1
print(dend.L,dend.diam,Rdend/1e6)
Rdend +=dend.L*1e-6/2.0/(dend.diam*1e-6/2.0)**2/pi*dend.Ra/100
Rdendf +=dend.L*1e-6/2.0/(dend.diam*1e-6/2.0)**2/pi*dend.Ra/100
print("The resistance of neck and the dendritic impedance are %.2f and %2.f MΩ (resp.)" %(Rneck/1e6,Rdend/1e6))

10.0 1.0
180.0 0.699999988079071 31.830988618379067
1.0 0.7 616.4818198894505
The resistance of neck and the dendritic impedance are 141.47 and 620 MΩ (resp.)


In [21]:
dendsizeL = max(max(max(dis['Dss'])+10,max(sis['Dss'])+10),177)
# Total sum is at least 180 um

In [22]:
#model.soma.diam = model.soma.L
dendN = model.dend[-1]
dendN.L = 1
dendp2 = dendN
dendp1 = dendc

In [23]:
dendsizeL = max(max(max(dis['Dss'])+10,max(sis['Dss'])+10),177)
dendp1.L = dendsizeL-10
dendp2.L = 9 
dendsh.L = 1
dendsh.diam = 0.7
dendc.diam = 0.7
dendN.diam = 0.7        

In [24]:
rhosp = 1.3
dendareaperL = rhosp*mean(sps['Ah'])+(pi*0.5)
factor = dendareaperL/(pi*0.5)
print(factor)
dendN.cm = 3.5
dendc.cm = 3.5

3.7487535482970595


In [25]:
dendp1.nseg = int(dendc.L//10)+1
dendp2.nseg = int(dendN.L//10)+1
print(dendp1.nseg,dendp2.nseg)
dendsh.nseg = 10
dendp1.nseg = 100
dendp2.nseg = 100
print(dendsh.nseg,dendp1.nseg,dendp2.nseg)

17 1
10 100 100


In [26]:
neck1 = model.neck[1]
sp1 = model.spne[1]

In [27]:
dendp2.name()


'DendN'

In [28]:
dendp1.nseg = 20
dendp2.nseg = 20
print(lb.h.topology())
dendp1.nseg = 100
dendp2.nseg = 100


|-|       soma(0-1)
   `|       dendA1c(0-1)
     `-------------------|       dendA1(0-1)
                          `---------|       DendE(0-1)
                                     `-------------------|       DendN(0-1)
                                `|       NSpne000(0-1)
                                  `|       Spne000(0-1)
                                `|       NSpne001(0-1)
                                  `|       Spne001(0-1)
   `|       dendB1(0-1)

1.0


In [29]:
neck1.connect(dendp1(0.5))
#sp1.connect(neck1(1))

NSpne001

## Plots

xkcd is to give it non serious appearance

In [30]:
gtrG = rhGABAR*30e-6*2/5.0
gtrA = rhAMPAR*11e-6*3/5.0
gtrN = rhNMDAR*30e-6*1.0/16.0/4.0/1.27*50*5

In [31]:
diam0 = 0.15
Rneck0 = Rneck/1e6
Cm0 = model.CM
print(Rneck0)

141.4710605261292


In [32]:
Rneck = neck.Ra*neck.L/(neck.diam)**2*0.04/pi
print(Rneck)

141.4710605261292


In [33]:
def simulateSet(model,spn,spn2,distance = 10, tG = 500,ton = 50,dton2 = 0,toffset = 50,t_stop = 250, EL = -65,btsr = None,VDCC = array([0.,0,0,0]), inh = 'in', silent = None, path='on'):
    if btsr is None:
        btsr = ones(9)==0
    model.E_PAS = EL
    model.soma.e_pas = model.E_PAS
    for dendp in model.dend:
        dendp.e_pas = model.E_PAS
    for sp in model.spne:
        sp.e_pas = model.E_PAS

    dendsh = model.dend[-2]
    dendc = model.dend[1]
    dendN = model.dend[-1]
    dendp2 = dendN
    dendp1 = dendc
    neck0 = model.neck[0]
    sp0 = model.spne[0]
    neck1 = model.neck[1]
    sp1 = model.spne[1]
    
    dshL = dendsh.L
    dendsizeL = dendsh.L+dendc.L+dendN.L
    data = column_stack((spn["A1"],spn["A1"],spn["A2"],spn["Rneck"],spn["Dss"],spn["L"],spn["D"],spn["Ln"],spn["AhA0"]))
    data2 = column_stack((spn2["A1"],spn2["Rneck"],spn2["L"],spn2["D"],spn2["Ln"],spn2["AhA0"]))
    if isnan(data).sum()>0:
        raise "There are nans do something about it"
    for i in range(9):
        if ~btsr[i]:
            data[:,i] = data[:,i].mean()
        
    
    mes = zeros((nsp,9))
    me2 = zeros((nsp,11))
    me3 = zeros((nsp,10))

    iad = 2
    vavg = zeros((int(t_stop/lb.h.dt)+iad,7))
    vtracs = zeros((int(t_stop/lb.h.dt)+iad,500))
    Ctracs = zeros((int(t_stop/lb.h.dt)+iad,500))
    vtracs2 = zeros((int(t_stop/lb.h.dt)+iad,500))
    Ctracs2 = zeros((int(t_stop/lb.h.dt)+iad,500))
    vtracsD = zeros((int(t_stop/lb.h.dt)+iad,500))
    vtracsS = zeros((int(t_stop/lb.h.dt)+iad,500))
    for i in arange(nsp):
        NC0.weight[0]  = data[i,0] *gtrA#/2
        NCN0.weight[0] = data[i,1] *gtrN#*0#*0
        
        
        if inh == 'in':
            NCG.weight[0] = data[i,2] *gtrG#*0
            NCGsh.weight[0] = 0.0*iPSDsh[i] *gtrG
        elif inh =='out':
            NCG.weight[0] = 0.0* data[i,2] *gtrG#*0
            NCGsh.weight[0] = iPSDsh[i] *gtrG
        else:
            NCG.weight[0] = data[i,2] *gtrG#*0
            NCGsh.weight[0] = iPSDsh[i] *gtrG
            
        NC1.weight[0]  = data2[i,0] *gtrA#/2
        NCN1.weight[0] = data2[i,0] *gtrN#*0#*0
        #NCG1.weight[0] = data2[i,0] *gtrG#*0
        if silent == 'sp0':
            NC0.weight[0] = 0
            NCN0.weight[0] = 0
        elif silent == 'sp1':
            NC1.weight[0]  = 0
            NCN1.weight[0] = 0
            
            
        neck0.L = data[i,7]  
        Rneck0 = neck.Ra*neck0.L/(diam0)**2*0.04/pi
        #print(Rneck0,data[i,3])
        neck0.diam = diam0*sqrt(Rneck0/data[i,3])
        
        neck1.L = data2[i,2]  
        Rneck1 = neck.Ra*neck1.L/(diam0)**2*0.04/pi
        #print(Rneck0,data[i,3])
        neck1.diam = diam0*sqrt(Rneck1/data[i,1])
        
        
        posD = data[i,4]
        
        dendc.L = posD-dshL/2
        dendN.L = max(dendsizeL-posD-dshL/2,2)

        
        # A = pi*D**2
        sp0.L = data[i,5]
        sp0.diam = data[i,6]
        sp0.cm = data[i,8]
        #spvol = sp0(0.5).volume()
        #spineArea =  sp0(0.5).area()#sp.L*sp.diam+1.8*sp.diam**2/4 # um^2
        
        CaTcond = 1e-3# pS
        sp0.pbar_caL13PS = VDCC[0]*CaTcond#/spineArea
        sp0.pbar_caLPS = VDCC[1]*CaTcond#/spineArea
        sp0.pbar_canPS = VDCC[2]*CaTcond#/spineArea
        sp0.pcaqbar_caqPS = VDCC[3]*CaTcond#/spineArea
        
        sp1.L = data2[i,2]
        sp1.diam = data2[i,3]
        sp1.cm = data2[i,5]
        
        CaTcond = 1e-3# pS
        sp1.pbar_caL13PS = VDCC[0]*CaTcond#/spineArea
        sp1.pbar_caLPS = VDCC[1]*CaTcond#/spineArea
        sp1.pbar_canPS = VDCC[2]*CaTcond#/spineArea
        sp1.pcaqbar_caqPS = VDCC[3]*CaTcond#/spineArea
        
        
        NC0.delay = toffset+ton-50
        NCN0.delay = toffset+ton-50
        NCG.delay = toffset+tG#toffset+tG#-50
        NCGsh.delay = toffset+tG#toffset+tG#-50

        NC1.delay = toffset+ton+dton2-50
        NCN1.delay = toffset+ton+dton2-50

        
        if path=='on':
            f = max(0.0,1-distance/dendp1.L)
            neck1.connect(dendp1(f))
        else:
            f = min(1.0,distance/dendp1.L)
            neck1.connect(dendp2(f))
            
        #if f >=0:
        #    neck1.connect(dendp1(f))
        #else:
        #    neck1.connect(dendp1(0.0))
        
        for il in range(10):
            f = 1-7.5*il/dendc.L
            #f = 0.5
            if f>=0:
                vDendEL[il].record(dendc(f)._ref_v)
            else:
                vDendEL[il].record(dendc(0.0)._ref_v)

        
        for il in range(10):
            f = 7.5*il/dendc.L
            #f = 0.5
            if f<=1:
                vDendEL2[il].record(dendN(f)._ref_v)
            else:
                vDendEL2[il].record(dendN(1.0)._ref_v)
                
        lb.h.finitialize(model.E_PAS)
        
        

                    
        #print(data[i,:])
        lb.neuron.run(t_stop)
        
        #plot(trec,vspneRec[0])    
        
        current = abs((array(vDendRec[1])-array(vrec[0]))/Rdend)
        
        vtracs[:,i] = array(vspneRec[0]) 
        vtracs2[:,i] = array(vspneRec[1]) 

        vtracsD[:,i] = array(vDendRec[1])
        #vtracsD[:,i] = array(vDendEL[0])
        vtracsS[:,i] = array(vrec) 

        vavg[:,0] += array(vspneRec[0]) 
        vavg[:,1] += array(vspneRec[0])**2
        vavg[:,2] += array(vDendRec[1]) 
        vavg[:,3] += array(vDendRec[1])**2
        vavg[:,4] += array(vrec) 
        vavg[:,5] += array(vrec)**2
        vavg[:,6] += 1
            
        cat = array(caDendRec['sp0 Ca'])/1e-3
        Ctracs[:,i] = cat-cat[0] 
        cat = array(caDendRec['sp1 Ca'])/1e-3
        Ctracs2[:,i] = cat-cat[0] 
        aG = abs(array(currentGABA)).argmax()
        aA = abs(array(currentAMPA)).argmax()
        
        mes[i,:] = [data[i,3],max(vspneRec[0])-EL,max(vDendRec[3])-EL,max(vrec)-EL,max(cat)-cat[0],array(currentGABA)[aG],array(currentAMPA)[aA],max(vspneRec[1])-EL,max(current)]
        me2[i,0] = max(vDendRec[3])
        me2[i,1:] = [max(vD) for vD in vDendEL]
        me3[i,:] = [max(vD) for vD in vDendEL2]

        #plot(trec,array(caDendRec[-1])/1e-3)
        #ylabel("[Ca] (uM)")
        #figure()
        #plot(trec,vspneRec[0])
        #break
        
    vavg[:,:5] = vavg[:,:5]/vavg[0,6]
    vavg[:,1] = sqrt(vavg[:,1]-vavg[:,0]**2)#/sqrt(vavg[0,6])
    vavg[:,3] = sqrt(vavg[:,3]-vavg[:,2]**2)#/sqrt(vavg[0,6])
    vavg[:,5] = sqrt(vavg[:,5]-vavg[:,4]**2)#/sqrt(vavg[0,6])
    return(vavg,mes,vtracs,vtracsD,vtracsS,Ctracs,me2,me3,array(trec),vtracs2,Ctracs2)

In [35]:
VDCCa =  array([0.04023183, 0.08000483, 0.03416801, 0.02747762] )
EL0 = -70

In [36]:
#vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sis,distance= 10, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
#plot(arange(vtracs0.shape[0])*lb.h.dt,vtracs0[:,2])
#plot(arange(vtracs0.shape[0])*lb.h.dt,vtracs2[:,2])
#plot(arange(vtracs0.shape[0])*lb.h.dt,vtracsD0[:,2])
#xlim(150,210)

In [37]:
from scipy.signal import find_peaks

In [38]:
def getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.5, plotose = False,):
    vt = 1.0*vtracs2b
    lpeaks = list(map(lambda x: (find_peaks(x-min(x),height=th)[1]['peak_heights'].max(), min(x)) , vt.transpose()))
    apeaks = array(lpeaks)
    apeaks[:,0] = apeaks[:,0]+apeaks[:,1]
    if plotose:
        hist(apeaks[:,0]-EL0,bins=51,label= 'baseline',alpha=0.7)
    ap0 = apeaks[:,0]
    vt = 1.0*vtracs2c
    lpeaks = list(map(lambda x: (find_peaks(x-min(x),height=th)[1]['peak_heights'].max(), min(x)) , vt.transpose()))
    apeaks = array(lpeaks)
    apeaks[:,0] = apeaks[:,0]+apeaks[:,1]
    if plotose:
        hist(apeaks[:,0]-EL0,bins=51,label= 'shaft inhibition',alpha=0.7)
    ap0 = column_stack((ap0,apeaks[:,0]))

    vt = 1.0*vtracs2
    lpeaks = list(map(lambda x: (find_peaks(x-min(x),height=th)[1]['peak_heights'].max(), min(x)) , vt.transpose()))
    apeaks = array(lpeaks)
    apeaks[:,0] = apeaks[:,0]+apeaks[:,1]
    if plotose:
        hist(apeaks[:,0]-EL0,bins=51,label= 'spinal inh',alpha=0.7)
        legend()
    ap0 = column_stack((ap0,apeaks[:,0]))
    ap0 = ap0

    return(ap0)

In [39]:
def getint(x):
    xc = x*1.0
    xc.sort()
    sh = xc.shape[0]
    xmed = xc[sh//2]
    s0= int(sh*(1-.6827)/2)
    s1 = sh-s0
    x0 = xc[s0]
    x1 = xc[s1]
    s0b= int(sh*(1-.95)/2)
    s1b = sh-s0b
    x0b = xc[s0b]
    x1b = xc[s1b]
    return((xmed,x0,x1,x0b,x1b))

In [44]:
dsv = array([1,2,3,4,5,6,7,8,9])*7.5

aps = {}

for i in range(50):
    dss = 0.0

    vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')

    ap00 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
    apS0 = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
    apD0 = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
    apC0 = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
    
    aps[('Vspine',dss,i)] = ap00
    aps[('Vsoma',dss,i)] = apS0
    aps[('Vdendrite',dss,i)] = apD0
    aps[('Ca',dss,i)] = apC0
    aps[('data',i)] = (dis,sis,iPSDsh)
    
    print(i,'0')
    with open("/mnt/data/gatinginfo_inh_temp.pickle","wb") as f:
        pickle.dump(aps,f)
    
    
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',-dss,i)] = ap0
        aps[('Vsoma',-dss,i)] = apS
        aps[('Vdendrite',-dss,i)] = apD
        aps[('Ca',-dss,i)] = apC
    print(i,'1')
    with open("/mnt/data/gatinginfo_inh_temp.pickle","wb") as f:
        pickle.dump(aps,f)
        
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0', path='off')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',dss,i)] = ap0
        aps[('Vsoma',dss,i)] = apS
        aps[('Vdendrite',dss,i)] = apD
        aps[('Ca',dss,i)] = apC
    
    dis = btset_cd(cd='DiS',size=nsp)
    sps = btset_cd(cd='Sp',size=nsp)

    iPSDsh = btShInh(size=500)
    
    dendsizeL = max(max(dis['Dss'])+10,177)
    dendp1.L = dendsizeL-10
    dendp2.L = 9 
    dendsh.L = 1    
    
    print(i,'2')
    with open("/mnt/data/gatinginfo_inh_temp.pickle","wb") as f:
        pickle.dump(aps,f)

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2
5 0
5 1
5 2
6 0
6 1
6 2
7 0
7 1
7 2
8 0
8 1
8 2
9 0
9 1
9 2
10 0
10 1
10 2
11 0
11 1
11 2
12 0
12 1
12 2
13 0
13 1
13 2
14 0
14 1
14 2
15 0
15 1
15 2
16 0
16 1
16 2
17 0
17 1
17 2
18 0
18 1
18 2
19 0
19 1
19 2
20 0
20 1
20 2
21 0
21 1
21 2
22 0
22 1
22 2
23 0
23 1
23 2
24 0
24 1
24 2
25 0
25 1
25 2
26 0
26 1
26 2
27 0
27 1
27 2
28 0
28 1
28 2
29 0
29 1
29 2
30 0
30 1
30 2
31 0
31 1
31 2
32 0
32 1
32 2
33 0
33 1
33 2
34 0
34 1
34 2
35 0
35 1
35 2
36 0
36 1
36 2
37 0
37 1
37 2
38 0
38 1


KeyboardInterrupt: 

In [45]:
with open("/mnt/data/gatinginfo_inh.pickle","wb") as f:
    pickle.dump(aps,f)

In [40]:
dsv = array([1,2,3,4,5,6,7,8,9])*7.5

aps = {}
dis = btset_cd(cd='DiS',size=nsp)
sps = btset_cd(cd='Sp',size=nsp)
dis['Rneck'] = dis['Rneck']*2
sps['Rneck'] = sps['Rneck']*2
for i in range(5):
    dss = 0.0

    vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')

    ap00 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
    apS0 = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
    apD0 = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
    apC0 = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
    
    aps[('Vspine',dss,i)] = ap00
    aps[('Vsoma',dss,i)] = apS0
    aps[('Vdendrite',dss,i)] = apD0
    aps[('Ca',dss,i)] = apC0
    aps[('data',i)] = (dis,sis,iPSDsh)
    
    print(i,'0')
    with open("/mnt/data/gatinginfo_inh_tempx2.pickle","wb") as f:
        pickle.dump(aps,f)
    
    
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',-dss,i)] = ap0
        aps[('Vsoma',-dss,i)] = apS
        aps[('Vdendrite',-dss,i)] = apD
        aps[('Ca',-dss,i)] = apC
    print(i,'1')
    with open("/mnt/data/gatinginfo_inh_tempx2.pickle","wb") as f:
        pickle.dump(aps,f)
        
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0', path='off')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',dss,i)] = ap0
        aps[('Vsoma',dss,i)] = apS
        aps[('Vdendrite',dss,i)] = apD
        aps[('Ca',dss,i)] = apC
    
    dis = btset_cd(cd='DiS',size=nsp)
    sps = btset_cd(cd='Sp',size=nsp)
    dis['Rneck'] = dis['Rneck']*2
    sps['Rneck'] = sps['Rneck']*2
    iPSDsh = btShInh(size=500)
    
    dendsizeL = max(max(dis['Dss'])+10,177)
    dendp1.L = dendsizeL-10
    dendp2.L = 9 
    dendsh.L = 1    
    
    print(i,'2')
    with open("/mnt/data/gatinginfo_inh_tempx2.pickle","wb") as f:
        pickle.dump(aps,f)

with open("/mnt/data/gatinginfo_inhx2.pickle","wb") as f:
    pickle.dump(aps,f)

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2


In [41]:
dsv = array([1,2,3,4,5,6,7,8,9])*7.5

aps = {}
dis = btset_cd(cd='DiS',size=nsp)
sps = btset_cd(cd='Sp',size=nsp)
dis['Rneck'] = dis['Rneck']*5
sps['Rneck'] = sps['Rneck']*5
for i in range(5):
    dss = 0.0

    vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')

    ap00 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
    apS0 = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
    apD0 = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
    apC0 = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
    
    aps[('Vspine',dss,i)] = ap00
    aps[('Vsoma',dss,i)] = apS0
    aps[('Vdendrite',dss,i)] = apD0
    aps[('Ca',dss,i)] = apC0
    aps[('data',i)] = (dis,sis,iPSDsh)
    
    print(i,'0')
    with open("/mnt/data/gatinginfo_inh_tempx5.pickle","wb") as f:
        pickle.dump(aps,f)
    
    
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',-dss,i)] = ap0
        aps[('Vsoma',-dss,i)] = apS
        aps[('Vdendrite',-dss,i)] = apD
        aps[('Ca',-dss,i)] = apC
    print(i,'1')
    with open("/mnt/data/gatinginfo_inh_tempx5.pickle","wb") as f:
        pickle.dump(aps,f)
        
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0', path='off')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',dss,i)] = ap0
        aps[('Vsoma',dss,i)] = apS
        aps[('Vdendrite',dss,i)] = apD
        aps[('Ca',dss,i)] = apC
    
    dis = btset_cd(cd='DiS',size=nsp)
    sps = btset_cd(cd='Sp',size=nsp)
    dis['Rneck'] = dis['Rneck']*5
    sps['Rneck'] = sps['Rneck']*5
    iPSDsh = btShInh(size=500)
    
    dendsizeL = max(max(dis['Dss'])+10,177)
    dendp1.L = dendsizeL-10
    dendp2.L = 9 
    dendsh.L = 1    
    
    print(i,'2')
    with open("/mnt/data/gatinginfo_inh_tempx5.pickle","wb") as f:
        pickle.dump(aps,f)

with open("/mnt/data/gatinginfo_inhx5.pickle","wb") as f:
    pickle.dump(aps,f)

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2


# Hyp case with same inh size

In [117]:
nsp = 500
iPSDsh = 1.0*dis['A2']
dsv = array([1,2,3,4,5,6,7,8,9])*7.5

aps = {}

for i in range(2):
    dss = 0.0

    vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
    vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')

    ap00 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
    apS0 = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
    apD0 = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
    apC0 = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
    
    aps[('Vspine',dss,i)] = ap00
    aps[('Vsoma',dss,i)] = apS0
    aps[('Vdendrite',dss,i)] = apD0
    aps[('Ca',dss,i)] = apC0
    aps[('data',i)] = (dis,sis,iPSDsh)
    
    print(i,'0')
    with open("/mnt/data/gatinginfo_inh_h_temp.pickle","wb") as f:
        pickle.dump(aps,f)
    
    
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',-dss,i)] = ap0
        aps[('Vsoma',-dss,i)] = apS
        aps[('Vdendrite',-dss,i)] = apD
        aps[('Ca',-dss,i)] = apC
    print(i,'1')
    with open("/mnt/data/gatinginfo_inh_h_temp.pickle","wb") as f:
        pickle.dump(aps,f)
        
    for dss in dsv:
        vavg0,mes0,vtracs0,vtracsD0,vtracsS0,Ctracs0,me20,me30, trec,vtracs2,Ctracs2 = simulateSet(model,dis,sps,distance = dss, tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0b,vtracsD0b,vtracsS0b,Ctracs0b,me20,me30, trec,vtracs2b,Ctracs2b = simulateSet(model,dis,sps,distance = dss,tG = 500,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'in',silent = 'sp0', path='off')
        vavg0,mes0,vtracs0c,vtracsD0c,vtracsS0c,Ctracs0c,me20,me30, trec,vtracs2c,Ctracs2c = simulateSet(model,dis,sps,distance = dss,tG = 10,ton = 50,dton2 = 15,toffset = 100,t_stop = 500, EL = -70,btsr = ones(9)==1,VDCC = VDCCa, inh = 'out',silent = 'sp0', path='off')
        
        ap0 = getvpeaks(vtracs2b,vtracs2,vtracs2c, th = 0.0, plotose = False)
        apS = getvpeaks(vtracsS0b,vtracsS0,vtracsS0c, th = 0.0, plotose = False)
        apD = getvpeaks(vtracsD0b,vtracsD0,vtracsD0c, th = 0.0, plotose = False)
        apC = getvpeaks(Ctracs2b,Ctracs2,Ctracs2c, th = 0., plotose = False)
        
        aps[('Vspine',dss,i)] = ap0
        aps[('Vsoma',dss,i)] = apS
        aps[('Vdendrite',dss,i)] = apD
        aps[('Ca',dss,i)] = apC
    
    dis = btset_cd(cd='DiS',size=nsp)
    sps = btset_cd(cd='Sp',size=nsp)

    iPSDsh = btShInh(size=500)
    
    dendsizeL = max(max(dis['Dss'])+10,177)
    dendp1.L = dendsizeL-10
    dendp2.L = 9 
    dendsh.L = 1    
    
    print(i,'2')
    with open("/mnt/data/gatinginfo_inh_h_temp.pickle","wb") as f:
        pickle.dump(aps,f)

0 0
0 1
0 2
1 0
1 1
1 2


In [118]:
with open("/mnt/data/gatinginfo_inh_h.pickle","wb") as f:
    pickle.dump(aps,f)

In [119]:
iPSDsh = btShInh(size=500)